# amazon 评论的情感识别
使用amazon的评论信息进行情感识别，目的是为了测试算法的识别准确度

因为我们当前的训练数据不靠谱所以拿amazon的评论数据来做算法效果验证

# Fasttext classification 

1. 格式转换工具将普通的csv训练数据文件变为共fasttext使用的训练数据

In [1]:
import pandas as pd
import os
import fasttext
import random

amazon_source = "../../datasets/amazon.csv"

def trans_file(old_file):
    x = old_file + ".train"
    y = old_file + ".valid"
    h = pd.read_csv(old_file)

    with open(x, "w+") as fx:
        with open(y, "w+") as fy:
            for i in range(0, len(h)):
                s = "__label__" + str(h['Positive'][i])+" " + h['reviewText'][i] + "\n"
                if i > 12000:
                    fy.write(s)
                else:
                    fx.write(s)
    return x,y

amazon_train, amazon_test = trans_file(amazon_source)

In [2]:
model = fasttext.train_supervised(input=amazon_train,epoch=5,lr=0.8,wordNgrams=2,dim=128,loss='hs')
model.test(amazon_test,k=-1)

Read 0M words
Number of words:  30628
Number of labels: 2
Progress: 100.0% words/sec/thread:  964496 lr:  0.000000 avg.loss:  0.111169 ETA:   0h 0m 0s


(7999, 0.5, 1.0)

In [3]:
model.predict("got this kindle fire for Christmas. trying to download free angry birds and it is not working. started to down load over 2 hours ago. an it is only at 2 percent still. piece of crap:(",k=-1)

(('__label__0', '__label__1'), array([0.9835794 , 0.01644061]))